In [12]:
source = open("05test.txt", "r").read()
sections = source.split("\n\n")

seed_list = sections[0];
seeds = seed_list.split(" ")[1:]
maps = sections[1:]

location_maps = []
for map in maps:
    lines = map.split("\n")
    label = lines[0]
    mappings = lines[1:]
    location_map = {"label": label, "mappings": []}

    for line in mappings:
        target_start, seed_start, seed_range = line.split(" ")
        seed_start = int(seed_start)
        target_start = int(target_start)
        seed_range = int(seed_range)
        location_map["mappings"].append((seed_start, target_start, seed_range))
    location_map["mappings"] = sorted(location_map["mappings"], key=lambda x: x[0])
    location_maps.append(location_map)

def apply_map(start, location_map):
    for mapping in location_map["mappings"]:
        seed_start, target_start, seed_range = mapping
        if start >= seed_start and start < seed_start + seed_range:
            return target_start + (start - seed_start)
    return start
ends = []
for seed in seeds:
    start = int(seed)
    for location_map in location_maps:
        # print("Starting on map: " + location_map["label"] + " with start: " + str(start) + "and mappings" + str(location_map["mappings"]))
        start = apply_map(start, location_map)
    # print("Ended with ", start)
    ends.append(start)
ends
min(ends)

35

In [13]:
location_maps

[{'label': 'seed-to-soil map:', 'mappings': [(50, 52, 48), (98, 50, 2)]},
 {'label': 'soil-to-fertilizer map:',
  'mappings': [(0, 39, 15), (15, 0, 37), (52, 37, 2)]},
 {'label': 'fertilizer-to-water map:',
  'mappings': [(0, 42, 7), (7, 57, 4), (11, 0, 42), (53, 49, 8)]},
 {'label': 'water-to-light map:', 'mappings': [(18, 88, 7), (25, 18, 70)]},
 {'label': 'light-to-temperature map:',
  'mappings': [(45, 81, 19), (64, 68, 13), (77, 45, 23)]},
 {'label': 'temperature-to-humidity map:',
  'mappings': [(0, 1, 69), (69, 0, 1)]},
 {'label': 'humidity-to-location map:',
  'mappings': [(56, 60, 37), (93, 56, 4)]}]

In [18]:
import numpy as np 
seed_ranges = (np.array_split([int(x) for x in seeds], 2))
print("starting with seed ranges", seed_ranges)
for location_map in location_maps:
    windows = []
    for seed_start, seed_range in seed_ranges:
        #evaluate if seed range is in location range
        seed_max = seed_start + seed_range
        # print(location_map["mappings"])
        for location_seed_start, location_start, location_range in location_map["mappings"]:
            # print("raw location info", location_seed_start, location_start, location_range)
            location_seed_max = location_seed_start + location_range
            location_offset = location_start - location_seed_start
            print("comparing seed range of", seed_start, seed_max, "with location range of", location_seed_start, location_seed_max, location_offset)
            #window possibilities
            if seed_start >= location_seed_start and seed_max <= location_seed_max:
                # seed is within the location window
                print("seed is inside location", seed_start, seed_max, "with location range of", location_seed_start, location_seed_max)
                windows.append((seed_start + location_offset, seed_range))
                print("added window of", [windows[-1]])
                break
            elif seed_start >= location_seed_start and seed_start < location_seed_max and seed_max > location_seed_max:
                # seed starts within the location window but ends outside
                print("seed starts within and ends outside location", seed_start, seed_max, "with location range of", location_seed_start, location_seed_max)
                windows.append((seed_start + location_offset, abs(seed_range - abs(seed_max - location_seed_max))))
                print("added window of", [windows[-1]])
            elif seed_start < location_seed_start and seed_max <= location_seed_max:
                print("seed starts before and ends  inside location", seed_start, seed_max, "with location range of", location_seed_start, location_seed_max)
                # seed starts outside the location window but ends within
                windows.append((location_offset, seed_range))
                print("added window of", [windows[-1]])
            elif seed_start < location_seed_start and seed_max > location_seed_max:
                print("seed starts end ends outside location", seed_start, seed_max, "with location range of", location_seed_start, location_seed_max)
                # seed starts and ends outside the location window
                windows.append((location_seed_start, location_range))
                print("added window of", [windows[-1]])
            else: 
                print("Total miss", seed_start, seed_max, "with location range of", location_seed_start, location_seed_max)
                windows.append((seed_start, seed_range)) # If I miss a window entirely that means theres no change to be had
                print("added window of", [windows[-1]])
    seed_ranges = list(set(windows))
    print("seed ranges at the end of " + location_map["label"], seed_ranges )
seed_ranges
print("seed ranges at the end", seed_ranges)
print("Start/stops at the end", [(x[0], x[0] + x[1] -1 ) for x in seed_ranges if x[1] > 0])
new_seeds = np.array([(x[0], x[0] + x[1] -1 ) for x in seed_ranges if x[1] > 0]).flatten()
print("new seeds", new_seeds)  

# new_ends = []
# # print(new_seeds)
# for seed in new_seeds:
#     start = int(seed)
# #     for location_map in location_maps:
# #         # print("Starting on map: " + location_map["label"] + " with start: " + str(start) + "and mappings" + str(location_map["mappings"]))
#     start = apply_map(start, location_maps[-1])
#     print("Ended with ", start)
#     new_ends.append(start)
# # new_ends
# min(new_ends)

starting with seed ranges [array([79, 14]), array([55, 13])]
comparing seed range of 79 93 with location range of 50 98 2
seed is inside location 79 93 with location range of 50 98
added window of [(81, 14)]
comparing seed range of 55 68 with location range of 50 98 2
seed is inside location 55 68 with location range of 50 98
added window of [(57, 13)]
seed ranges at the end of seed-to-soil map: [(81, 14), (57, 13)]
comparing seed range of 81 95 with location range of 0 15 39
seed starts within and ends outside location 81 95 with location range of 0 15
added window of [(120, 66)]
comparing seed range of 81 95 with location range of 15 52 -15
seed starts within and ends outside location 81 95 with location range of 15 52
added window of [(66, 29)]
comparing seed range of 81 95 with location range of 52 54 -15
seed starts within and ends outside location 81 95 with location range of 52 54
added window of [(66, 27)]
comparing seed range of 57 70 with location range of 0 15 39
seed starts